# Imports

In [25]:
import  tensorflow                  as      tf
import  numpy                       as      np
import  os
from    sklearn.model_selection     import  train_test_split
from    tensorflow.keras.utils      import  to_categorical

from    tensorflow.keras.models     import  Sequential
from    tensorflow.keras.layers     import  LSTM, Dense
from    tensorflow.keras.callbacks  import  TensorBoard

## Constants

In [29]:
SAMPLES_PER_WORD = 60
FRAMES_PER_SAMPLE = 30

SAMPLES_PATH = "/Volumes/SSD/collection/"

In [34]:
words = np.array(['hello', 'my'])
words_path = {
    'hello' : 1,
    'my' : 5
    }
labels_map = {label:num for num, label in enumerate(words)}
labels_map

{'hello': 0, 'my': 1}

In [39]:
a = np.load("/Volumes/SSD/collection/5/5_4_24.npy")
a
len(a)

samples, labels = [], []
for word in words:
    for sample in range(SAMPLES_PER_WORD):
        sample_data = []
        for frame in range(FRAMES_PER_SAMPLE):
            path        = os.path.join(SAMPLES_PATH, f'{words_path[word]}', f'{words_path[word]}_{sample}_{frame}.npy')
            frame_data  = np.load(path)
            sample_data.append(frame_data)
        samples.append(sample_data)
        labels.append(labels_map[word])

In [51]:
X = np.array(samples)

In [46]:
y = to_categorical(labels).astype(int)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Model

In [50]:
model           = Sequential()

lstm_model_1    = LSTM(     64,     return_sequences = True,    activation = "relu", input_shape=(30, 1629) )
lstm_model_2    = LSTM(     128,    return_sequences = True,    activation = "relu" )
lstm_model_3    = LSTM(     64,     return_sequences = False,   activation = "relu" )

dense_model_1   = Dense(    64,                                 activation = "relu" )
dense_model_2   = Dense(    32,                                 activation = "relu" )
dense_model_3   = Dense(    words.shape[0],                     activation = "softmax" )

model.add( lstm_model_1 )
model.add( lstm_model_2 )
model.add( lstm_model_3 )

model.add( dense_model_1 )
model.add( dense_model_2 )
model.add( dense_model_3 )

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [52]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000)